In [36]:
import pandas as pd

path = "/Users/ichida/dev/datasets/zara_challenge/" \
       "zara_data_go_2019_all_dataset"

sales_stock_df = pd.read_csv(f"{path}/sales_stock.csv")
products_df = pd.read_csv(f"{path}/products.csv")
positions_df = pd.read_csv(f"{path}/positions.csv")
blocks_df = pd.read_csv(f"{path}/product_blocks.csv")

### Retrieve the following features:
- Quantity of items in each block

In [37]:
grouped_df = blocks_df.groupby('block_id').count()['product_id'].reset_index()
grouped_df.columns = ['block_id', 'qtty_items']
features_df = pd.merge(blocks_df, grouped_df, on='block_id', how='inner')
features_df = pd.merge(products_df, features_df, on='product_id', how='left')

In [38]:
sales_stock_sum = sales_stock_df.groupby(['date_number', 'product_id']).agg({'sales':'sum', 'stock':'sum'}).reset_index()
sales_stock_sum_63 = sales_stock_sum[sales_stock_sum['date_number'] <= 63]
count_categories_per_product = positions_df.groupby(['date_number', 'product_id']).count()['position'].reset_index()
position_features = positions_df.groupby(['date_number', 'product_id']).agg({'position':['max', 'mean', 'min']}).reset_index()
# Including feature columns
sales_stock_features = pd.merge(sales_stock_sum_63, features_df, on='product_id')
sales_stock_features = pd.merge(sales_stock_features, count_categories_per_product, on=['date_number', 'product_id'])
sales_stock_features = pd.merge(sales_stock_features, position_features, on=['date_number', 'product_id'])

C:\Users\ichida\anaconda3\envs\ml\lib\site-packages\pandas\core\reshape\merge.py:618: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\ichida\anaconda3\envs\ml\lib\site-packages\pandas\core\generic.py:3936: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


In [39]:
# Calculating the daily revenue
sales_stock_features.loc[:, 'revenue'] = sales_stock_features['sales'] * sales_stock_features['price']
# Get total revenue per block
blocks_revenue = sales_stock_features.groupby(['block_id']).sum()['revenue'].reset_index()
blocks_revenue.columns = ['block_id', 'total_revenue']
sales_stock_features = pd.merge(sales_stock_features, blocks_revenue, on='block_id')

In [40]:
import math
blocks_revenue = pd.merge(blocks_revenue, grouped_df, on='block_id')
blocks_revenue = blocks_revenue.sort_values(ascending=False, by='total_revenue').reset_index()
df2=pd.DataFrame()
for k in range(1,11):
    df2=pd.concat([df2, blocks_revenue[blocks_revenue['qtty_items']==k].head(math.floor(50/k))])

df2=df2.sort_values(by='total_revenue', ascending=False).reset_index().drop('index', axis=1)
most_revenues=df2.copy()

In [42]:
from knapsack import knapsack
import time

weights = tuple(df2['qtty_items'][0:40])
values = tuple(df2['total_revenue'][0:40])
capacity = 50
t0=time.time()
sol=knapsack(weights, values).solve(capacity)

checked_revenue=0
items=0
indices=sol[1]
chosen_blocks=[]
for x in indices:
    chosen_blocks.append(df2.loc[x,'block_id'].item())
    checked_revenue+=df2.loc[x,'total_revenue'].item()
    items+=df2.loc[x,'qtty_items'].item()
print('Number of blocks checked: {} out of {}'.format(70, len(df2)))
print('Number of blocks chosen:', len(chosen_blocks))
print('Chosen blocks list:', chosen_blocks)
print('Checked total revenue:', round(checked_revenue, 3))
print('Checked number of items:', items)
print('Execution time: {} sec.'.format(round(time.time()-t0, 2)))


Number of blocks checked: 70 out of 144
Number of blocks chosen: 11
Chosen blocks list: [2128, 2037, 9, 2503, 2306, 2481, 1256, 921, 753, 1080, 897]
Checked total revenue: 1843886.85
Checked number of items: 50
Execution time: 13.74 sec.


In [35]:
df3 = df2[df2['block_id'].isin(chosen_blocks)]

